# Encuesta continua de hogares 2020

### Cuánto afecta el nivel eductativo y la cantidad de hijos a las posibilidades de inserción laboral de una mujer

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(12,8)})

In [ ]:
df = pd.read_spss('HyP_2020_Terceros.sav')

df = df.rename(columns={
    'numero': 'id_hogar',
    'e26': 'sexo',
    'e27': 'edad',
    'e563': 'genero',
    'e29_6': 'ascendencia',
    'e30': 'parentesco',
    'pobpcoac': 'estado_laboral',
    'g126_1': 'sueldo',
    'f99': 'busca_otro_trabajo',
    'f72_2': 'actividad'
    ''
    })

df = df[[
    'id_hogar', 
    'sexo', 
    'edad', 
    'genero', 
    'ascendencia', 
    'parentesco', 
    'estado_laboral', 
    'nper', 
    'sueldo', 
    'busca_otro_trabajo',
    'actividad'
    ]]

df['nper'] = df['nper'].astype('category')
df['sexo'] = df['sexo'].apply(lambda x:'Masculino' if x == 1 else 'Femenino')

In [ ]:
df.sample(5)

In [ ]:
df.info()

In [ ]:
df.describe().transpose()

In [ ]:
df['estado_laboral'].value_counts() / len(df['estado_laboral']) * 100

## Género

In [ ]:
df['genero'].value_counts()

## Ascendencia

In [ ]:
asc_princ_porcentage = df['ascendencia'].value_counts() / len(df['ascendencia']) * 100

asc_princ_porcentage

## Composición del hogar

In [ ]:
df['parentesco'].value_counts() 

## Habitantes del hogar

In [ ]:
familia = pd.pivot_table(df, index='id_hogar', columns='parentesco',values=['nper'], aggfunc='count')

familia['totales'] = familia.apply(lambda x:x.sum(), axis=1)

familia = familia.sort_values(by='totales', ascending=False)

familia['totales'].value_counts()

## Cantidad de habitantes por hogar

In [ ]:
df['parentesco_num'] = df['parentesco'].cat.codes

df[['parentesco_num','parentesco']].value_counts()

In [ ]:
df[['sexo']][df['parentesco'].str.contains('Jefe')].value_counts().plot(kind=('bar'))

## Madres

In [ ]:
mask = df['parentesco_num'].isin([3,5,6])
df = df[mask]

df['es_madre'] = np.NaN
df['es_madre'] = df['es_madre'].apply(lambda x:'madre').where(df['parentesco_num'] == 6).where(df['sexo'] == 'Femenino')

## Eliminar filas de hombres jefes de hogar

In [ ]:
df_madre_y_familia = df.copy().drop(labels=list(df.loc[(df['sexo'].str.contains('Masculino')) & (df['parentesco_num'] == 6)].index))

df_madre_y_familia


In [ ]:
df_madre_y_familia['parentesco'].value_counts()

## Composición de familia solo con jefa de hogar

In [ ]:
df_madre_y_familia['hijos'] = df_madre_y_familia.groupby('id_hogar')['id_hogar'].transform('count') - 1

df_madre_y_familia

## Cantidad de hijos

In [ ]:
df_madre_y_familia.where(df['es_madre'].str.contains('madre'))['hijos'].value_counts()

In [ ]:
df_moms = df_madre_y_familia.copy()[df['es_madre'].notna()]

df_moms

In [ ]:
sns.boxplot(data=df_moms, x='hijos', y='sueldo')

plt.show()